<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Deploy a Python Script that uses COS connected data</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
</table>

This notebook demonstrates how to deploy a python script which makes uses of connected data from Cloud Object Storage (COS). You will upload data to a COS bucket and create a connection using your COS credentials, as well as a connected data asset with the uploaded data. You'll learn how to save and deploy a Python script in the deployment space – when this script is run, it will use the COS connected data and store the results in the deployment space.

## Table of Contents
1. [Setting up](#setup) <br>
2. [Create a Cloud Object Storage (COS) connection](#cos)<br>
    2.1 [Create a COS bucket and upload data](#cos_bucket)<br>
    2.2 [Create a connection](#connection)
3. [Create a Python script](#script)
4. [Create connected data assets](#data_asset)<br>
    4.1 [Create the source connected data](#source_data)<br>
    4.2 [Create destination connected data](#destination_data)
5. [Deploy the Python script](#deploy)<br>
    5.1 [Create a batch deployment](#deploy_script)<br>
    5.2 [Run the script](#run_script)<br>
    5.3 [Cleaning up](#delete)
6. [Summary and next steps](#summary)

To get started on CP4D 3.0, find documentation on installation and set up <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener no referrer">here</a>.

## 1. Setting up <a id="setup"></a>

Import the `watson-machine-learning-client` module.
<div class="alert alert-block alert-info">
For more information about the <b>Watson Machine Learning Python client (V4)</b>, please refer to the <a href="https://wml-api-pyclient-dev-v4.mybluemix.net/" target="_blank" rel="noopener no referrer">Python client documentation</a>. If you're using the notebook within a project on your CP4D cluster, the package comes pre-installed with the notebooks – but you can update the package below.
</div>

In [ ]:
!pip install --upgrade watson-machine-learning-client-V4

In [1]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**Authenticate the Python client on CP4D.**

<div class="alert alert-block alert-info">To find your authentication information (your credentials) follow the steps provided here in the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-authentication-local.html" target="_blank" rel="noopener no referrer">Documentation.</a></div>

**Action**: Enter your credentials in the following cell.

In [2]:
wml_credentials = {
    "url": "---",
    "username": "---",
    "password": "---",
    "instance_id": "wml_local",
    "version": "3.0.0"
}

In [3]:
client = WatsonMachineLearningAPIClient(wml_credentials)

### To set the default space, follow these steps.

<div class="alert alert-block alert-info">
You can create your own <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-spaces_local.html" target="_blank" rel="noopener no referrer">deployment space</a> by selecting <b>Analytics deployments</b> under <b>Analyze</b> from the Navigation Menu on the top left of this page.</div>

Alternatively, you can create a deployment using the code in the following cell. The cell is not executable cell at this stage, but you can enter the name of your space in the metadata and use it if needed.

In [4]:
# Obtain the UId of your space
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['guid'])

**Action:** Enter the name of your deployment space in the code below: `space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')`.

In [5]:
# Enter the name of your deployment space here:
space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')
print("Space UID = " + space_uid)

Space UID = 219be37a-7417-4bf0-9d8f-584bc3dbc468


You can set the default space using the cell below.

In [6]:
client.set.default_space(space_uid)

'SUCCESS'

## 2. Create a Cloud Object Storage (COS) connection <a id="cos"></a>

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/cloud-object-storage" target="_blank" rel="noopener no referrer">Cloud Object Storage (COS)</a> instance (a lite plan is offered and information about how to order storage is <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml_dlaas_object_store.html?audience=wdp" target="_blank" rel="noopener no referrer">here</a>). 
- Create new credentials with HMAC: 
    - Go to your COS dashboard.
    - In the **Service credentials** tab, click **New Credential+**.
    - Add the inline configuration parameter: {"HMAC":true}, click **Add**. (For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml_dlaas_object_store.html#createcredentials" target="_blank" rel="noopener no referrer">HMAC</a>.)

    This configuration parameter adds the following section to the instance credentials, (for use later in this notebook):
    ```
      "cos_hmac_keys": {
            "access_key_id": "-------",
            "secret_access_key": "-------"
       }
    ```


### 2.1 Create a COS bucket and upload data <a id="cos_bucket"></a>

In [7]:
# Import the boto library
import ibm_boto3
from ibm_botocore.client import Config
import os

Authenticate to COS and define the endpoint you will use.

1. Enter your COS credentials in the following cell. You can find these credentials in your COS instance dashboard under the **Service credentials** tab as described above.

2. Go to the Endpoint tab in the COS instance's dashboard to get the endpoint information, for example: `s3-api.us-geo.objectstorage.softlayer.net`. Enter it in the cell containing the variable `service_endpoint`.

In [8]:
# Enter your COS credentials.
cos_credentials = {
  "apikey": "***",
  "cos_hmac_keys": {
    "access_key_id": "***",
    "secret_access_key": "***"
  },
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "***",
  "iam_apikey_name": "***",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "***",
  "resource_instance_id": "***"
}

In [10]:
# Enter your Endpoint information.
service_endpoint = "https://s3.us-south.cloud-object-storage.appdomain.cloud"

Create the Boto resource.

In [12]:
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint='https://iam.bluemix.net/oidc/token',
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

Create the buckets you will use to store training data and training results.

**Note:** Bucket names must be unique.

In [13]:
# Create a bucket
from uuid import uuid4
bucket = 'script-' + str(uuid4())

if not cos.Bucket(bucket) in cos.buckets.all():
    print('Creating bucket "{}"...'.format(bucket))
    try:
        cos.create_bucket(Bucket=bucket)
    except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
        print('Error: {}.'.format(e.response['Error']['Message']))

Creating bucket "script-30e8e652-248e-4125-b342-70c06ce2612f"...


Check that the bucket was created.

In [14]:
print([x.name for x in list(cos.buckets.all()) if x.name.startswith('script')])

['script-30e8e652-248e-4125-b342-70c06ce2612f']


Use the following code to download the Tensorflow input .zip file from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.

In [ ]:
filename='tensorflow_input.zip'
!wget https://github.com/IBMDataScience/sample-notebooks/raw/master/Files/tensorflow_input.zip --output-document=tensorflow_input.zip

In [16]:
with open(filename, "rb") as data:
    cos.Bucket(bucket).upload_fileobj(data, filename)
    print('{} is uploaded.'.format(filename))

tensorflow_input.zip is uploaded.


### 2.2 Create a connection <a id="connection"></a>

You'll use the COS credentials and endpoint information to create the connection metadata. You need to choose the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/manage-data/conn_types.html" target="_blank" rel="noopener no referrer">connection type</a> as Cloud Object Storage as shown in the code below.

In [17]:
# Get the UID of the COS type connection.
data_source_type_id = client.connections.get_datasource_type_uid_by_name('cloudobjectstorage')

# create the metadata for the connector.
conn_meta_props = {
    client.connections.ConfigurationMetaNames.NAME: 'New Connection using the Python client',
    client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: data_source_type_id,
    client.connections.ConfigurationMetaNames.DESCRIPTION: 'Create a connection object using a COS instance',
    client.connections.ConfigurationMetaNames.PROPERTIES: {
        'access_key': cos_credentials['cos_hmac_keys']['access_key_id'],
        'iam_url': 'https://iam.cloud.ibm.com/identity/token',  # used when getting assets from this connection
        'secret_key': cos_credentials['cos_hmac_keys']['secret_access_key'],
        'url': service_endpoint
    }
}

In [18]:
conn_details = client.connections.create(meta_props=conn_meta_props)

Creating connections...
SUCCESS


Get the connection UID.

In [19]:
conn_uid = client.connections.get_uid(conn_details)
print('Connection UID: {}'.format(conn_uid))

Connection UID: 5b42ad4c-4163-4dd4-93de-d2d57b6b0f9b


## 3. Create a Python script <a id="script"></a>

First, download the python script file from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.

In [ ]:
!wget https://github.com/IBMDataScience/sample-notebooks/raw/master/Files/test_cos_script.py --output-document=test_cos_script.py

You'll use the following code to input your credentials in the file.

In [21]:
# Read the file (python script)
with open('test_cos_script.py', 'r') as file:
    filedata = file.read()

# Replace placeholder values with your credentials
filedata = filedata.replace('**YOUR URL**', wml_credentials["url"]).replace(
    '**YOUR USERNAME**', wml_credentials["username"]).replace('**YOUR PASSWORD**', wml_credentials["password"])

# Write the file out again
with open('test_cos_script.py', 'w') as file:
    file.write(filedata)

Create the metadata for the python script.

In [22]:
# Get the UID of the software specification.
sw_spec_uid = client.software_specifications.get_uid_by_name("default_py3.6")

# Create the metadata for the script
script_meta = {
    client.script.ConfigurationMetaNames.NAME: "test_script_cos.py",
    client.script.ConfigurationMetaNames.DESCRIPTION: "test python file",
    client.script.ConfigurationMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid
}

<div class="alert alert-block alert-info">To list the supported software specifications, run <tt>client.software_specifications.list()</tt>.<br>To find more information about the frameworks with their respective <b>Types</b> and <b>Software Specifications</b>, visit the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/wmls/wmls-deploy-python-types.html" target="_blank" rel="noopener no referrer">documentation</a>.</div>

Store the script in the deployment space.

In [23]:
script_file = 'test_cos_script.py'
script_details = client.script.store(meta_props=script_meta, file_path=script_file)

Creating Script asset...
SUCCESS


In [58]:
client.script.list()

------------------  ----------  ------------------------------------
NAME                ASSET_TYPE  ASSET_ID
test_script_cos.py  script      1086566b-020f-447b-86f8-7e92745300c5
------------------  ----------  ------------------------------------


<div class="alert alert-block alert-info">
From the list of scripts, you can see that the script was successfully saved in the deployment space. </div>

Get the UID of the stored script.

In [24]:
script_uid = client.script.get_uid(script_details)
print("Script UID: {}".format(script_uid))

Script UID: 00207fca-5927-477d-a956-dd72beca84a4


## 4. Create connected data assets <a id="data_asset"></a>

### 4.1 Create the source connected data <a id="source_data"></a>

Use the following code to save the Tensorflow input file as a .zip file and add it as a data asset in your deployment space. You'll use the connection UID and point to where the file is located in the COS bucket in the data asset metadata.

In [25]:
# Create the metadata for the data asset.
file_location = bucket+'/'+filename

asset_meta_props = {
    client.data_assets.ConfigurationMetaNames.NAME: 'TF_input.zip',
    client.data_assets.ConfigurationMetaNames.CONNECTION_ID: conn_uid,
    client.data_assets.ConfigurationMetaNames.DESCRIPTION: 'Connection using COS instance',
    # This is the data will be stored by WML.
    client.data_assets.ConfigurationMetaNames.DATA_CONTENT_NAME: file_location
}

In [26]:
# Store the asset and upload the data.
asset_details = client.data_assets.store(asset_meta_props)

Creating data asset...
SUCCESS


In [27]:
asset_details

{'metadata': {'space_id': '219be37a-7417-4bf0-9d8f-584bc3dbc468',
  'guid': 'e04f0c80-a826-40f4-b0f1-b857b05d8676',
  'href': '/v2/assets/e04f0c80-a826-40f4-b0f1-b857b05d8676?space_id=219be37a-7417-4bf0-9d8f-584bc3dbc468',
  'asset_type': 'data_asset',
  'created_at': '2020-05-04T21:00:45Z',
  'last_updated_at': '2020-05-04T21:00:45Z'},
 'entity': {'data_asset': {'mime_type': 'application/zip'}}}

You can see the connected data asset is stored in your deployment space.

In [28]:
client.data_assets.list()

------------  ----------  ----  ------------------------------------
NAME          ASSET_TYPE  SIZE  ASSET_ID
TF_input.zip  data_asset  0     e04f0c80-a826-40f4-b0f1-b857b05d8676
------------  ----------  ----  ------------------------------------


Retrieve the URL and UID of the stored asset.

In [29]:
src_asset_href = client.data_assets.get_href(asset_details)

In [30]:
src_asset_uid = client.data_assets.get_uid(asset_details)
print("Source asset UID: {}".format(src_asset_uid))

Source asset UID: e04f0c80-a826-40f4-b0f1-b857b05d8676


### 4.2 Create destination connected data <a id="destination_data"></a>

Use the following code to add a data asset to your deployment space where the output of the script will be saved.

In [31]:
# Create the metadata for the data asset.
file_location = bucket+'/script_output.zip'

dest_asset_meta_props = {
    client.data_assets.ConfigurationMetaNames.NAME: 'sample_output.zip',
    client.data_assets.ConfigurationMetaNames.CONNECTION_ID: conn_uid,
    client.data_assets.ConfigurationMetaNames.DESCRIPTION: 'Connection using COS instance',
    # This is the data will be stored by WML.
    client.data_assets.ConfigurationMetaNames.DATA_CONTENT_NAME: file_location
}

In [32]:
# Store the destination data asset
dest_asset_details = client.data_assets.store(dest_asset_meta_props)

Creating data asset...
SUCCESS


In [33]:
dest_asset_details

{'metadata': {'space_id': '219be37a-7417-4bf0-9d8f-584bc3dbc468',
  'guid': 'a5d70568-5da1-437c-8645-c69cd08fe057',
  'href': '/v2/assets/a5d70568-5da1-437c-8645-c69cd08fe057?space_id=219be37a-7417-4bf0-9d8f-584bc3dbc468',
  'asset_type': 'data_asset',
  'created_at': '2020-05-04T21:00:49Z',
  'last_updated_at': '2020-05-04T21:00:49Z'},
 'entity': {'data_asset': {'mime_type': 'application/zip'}}}

You can see the connected data asset is stored in your deployment space.

In [34]:
client.data_assets.list()

-----------------  ----------  ----  ------------------------------------
NAME               ASSET_TYPE  SIZE  ASSET_ID
TF_input.zip       data_asset  0     e04f0c80-a826-40f4-b0f1-b857b05d8676
sample_output.zip  data_asset  0     a5d70568-5da1-437c-8645-c69cd08fe057
-----------------  ----------  ----  ------------------------------------


Retrieve the URL and UID of the stored asset.

In [35]:
dest_asset_href = client.data_assets.get_href(dest_asset_details)

In [36]:
dest_asset_uid = client.data_assets.get_uid(dest_asset_details)
print("Destination asset UID: {}".format(dest_asset_uid))

Destination asset UID: a5d70568-5da1-437c-8645-c69cd08fe057


## 5. Deploy the Python script <a id="deploy"></a>

### 5.1 Create a batch deployment <a id="deploy_script"></a>

<a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/deploy-script.html" target="_blank" rel="noopener no referrer">Deploy the Python script</a> in the deployment space – this should be a <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/deploy-batch-jobs.html" target="_blank" rel="noopener no referrer">batch deployment</a>. This will process the input file using the script and produce the output file.

In [37]:
# Script deployment metadata.
script_deploy_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME:"Script Batch with COS Asset",
    client.deployments.ConfigurationMetaNames.BATCH:{},
    client.deployments.ConfigurationMetaNames.COMPUTE:{"name":"S","nodes":1}
}

In [38]:
script_deployment = client.deployments.create(artifact_uid=script_uid, meta_props=script_deploy_meta_props)



#######################################################################################

Synchronous deployment creation for uid: '00207fca-5927-477d-a956-dd72beca84a4' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='8501cb10-c2d3-424c-8807-e4e34fe30398'
------------------------------------------------------------------------------------------------




In [39]:
client.deployments.list()

------------------------------------  ---------------------------  -----  ------------------------  -------------
GUID                                  NAME                         STATE  CREATED                   ARTIFACT_TYPE
8501cb10-c2d3-424c-8807-e4e34fe30398  Script Batch with COS Asset  ready  2020-05-04T21:00:51.946Z  unknown
------------------------------------  ---------------------------  -----  ------------------------  -------------


<div class="alert alert-block alert-info">
From the list of deployments, you can see that the script was successfully deployed in the deployment space.</div>

Get the script deployment details.

In [40]:
script_deployment

{'metadata': {'parent': {'href': ''},
  'name': 'Script Batch with COS Asset',
  'guid': '8501cb10-c2d3-424c-8807-e4e34fe30398',
  'description': '',
  'id': '8501cb10-c2d3-424c-8807-e4e34fe30398',
  'modified_at': '2020-05-04T21:00:51.946Z',
  'created_at': '2020-05-04T21:00:51.946Z',
  'href': '/v4/deployments/8501cb10-c2d3-424c-8807-e4e34fe30398',
  'space_id': '219be37a-7417-4bf0-9d8f-584bc3dbc468'},
 'entity': {'name': 'Script Batch with COS Asset',
  'custom': {},
  'description': '',
  'compute': {'name': 'S', 'nodes': 1},
  'batch': {},
  'space': {'id': '219be37a-7417-4bf0-9d8f-584bc3dbc468',
   'href': '/v4/spaces/219be37a-7417-4bf0-9d8f-584bc3dbc468'},
  'status': {'state': 'ready'},
  'asset': {'id': '00207fca-5927-477d-a956-dd72beca84a4',
   'href': '/v2/assets/00207fca-5927-477d-a956-dd72beca84a4?space_id=219be37a-7417-4bf0-9d8f-584bc3dbc468'},
  'space_id': '219be37a-7417-4bf0-9d8f-584bc3dbc468'}}

Get the UID of the script deployment.

In [41]:
script_deployment_uid = client.deployments.get_uid(script_deployment)
print("Script deployment UID: {}".format(script_deployment_uid))

Script deployment UID: 8501cb10-c2d3-424c-8807-e4e34fe30398


### 5.2 Run the script <a id="run_script"></a>

Create the job which will run the script.

In [42]:
# Job payload.
script_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA_REFERENCES: [{
        "name": "script_ref_input",
        "type": "data_asset",
        "connection": {},
        "location": {
            "href": src_asset_href
        },
    }],
    client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE: {
        "name": "script_ref_output",
        "type": "data_asset",
        "location": {
            "href": dest_asset_href
            }
        }
    }
from pprint import pprint
pprint(script_payload)

{'input_data_references': [{'connection': {},
                            'location': {'href': '/v2/assets/e04f0c80-a826-40f4-b0f1-b857b05d8676?space_id=219be37a-7417-4bf0-9d8f-584bc3dbc468'},
                            'name': 'script_ref_input',
                            'type': 'data_asset'}],
 'output_data_reference': {'location': {'href': '/v2/assets/a5d70568-5da1-437c-8645-c69cd08fe057?space_id=219be37a-7417-4bf0-9d8f-584bc3dbc468'},
                           'name': 'script_ref_output',
                           'type': 'data_asset'}}


Use the following method to run and poll the job.

In [43]:
job = client.deployments.create_job(deployment_id=script_deployment_uid, meta_props=script_payload)
script_job_uid = client.deployments.get_job_uid(job)
print('Job UID: {}'.format(script_job_uid))

Job UID: 3146a408-c333-43a1-b148-40b0386d1f71


In [44]:
def poll_async_job(client, job_uid):
    import time
    while True:
        job_status = client.deployments.get_job_status(job_uid)
        print(job_status)
        state = job_status['state']
        if state == 'completed' or 'fail' in state:
            return client.deployments.get_job_details(job_uid)
        time.sleep(5)

In [45]:
# Display the result.
job_details = poll_async_job(client, script_job_uid)
pprint(job_details)

{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'running', 'running_at': '2020-05-04T21:01:28.033Z', 'completed_at': ''}
{'state': 'running', 'running_at': '2020-05-04T21:01:28.033Z', 'completed_at': ''}
{'state': 'running', 'running_at': '2020-05-04T21:01:28.033Z', 'completed_at': ''}
{'state': 'running', 'running_at': '2020-05-04T21:01:28.033Z', 'completed_at': ''}
{'state': 'running', 'running_at': '2020-05-04T21:01:28.033Z', 'completed_at': ''}
{'state': 'completed', 'message': {'text': 'The directory pointed by the environment variable BATCH_OUTPUT_DIR is empty, skipping content upload to data asset', 'level': 'warning'}, 'running_at': '2020-05-04T21:01:28.011119Z', 'completed_at': '2020-05-04T21:01:56.144664Z'}
{'entity': {'deployment': {'href': '/v4/deployments/8501cb10-c2d3-424c-8807-e4e

Get the UID of the data asset and download the results.

In [46]:
import re
output_href = job_details['entity']['scoring']['output_data_reference']['location']['href']
output_uid = re.split('[?/]', output_href)[3]
print('Script results UID = {}'.format(output_uid))

Script results UID = a5d70568-5da1-437c-8645-c69cd08fe057


In [47]:
client.data_assets.download(output_uid, filename='output.zip')

Successfully saved data asset content to file: 'output.zip'


'/home/wsuser/work/output.zip'

In [48]:
!unzip output.zip

Archive:  output.zip
  inflating: data_0.json             
  inflating: data_1.json             
  inflating: data_2.json             
  inflating: data_3.json             


### 5.3 Cleaning up <a id="delete"></a>

You can clean up your deployment space with the following lines of code.

In [49]:
client.deployments.delete_job(script_job_uid)

'SUCCESS'

In [50]:
client.deployments.delete(script_deployment_uid)

'SUCCESS'

In [51]:
client.script.delete(script_uid)

'SUCCESS'

In [52]:
client.data_assets.delete(src_asset_uid)

'SUCCESS'

In [53]:
client.data_assets.delete(dest_asset_uid)

'SUCCESS'

In [54]:
client.connections.delete(conn_uid)

'SUCCESS'

## 6. Summary <a id="summary"></a>

You successfully completed this notebook! 

You learned how to use Watson Machine Learning to create and deploy a python script that uses COS connected data – you learned how to create a batch deployment, a connection, and a connected data asset using Watson Machine Learning.

### Resources <a id="resources"></a>

To learn more about configurations used in this notebook or more sample notebooks, tutorials, documentation, how-tos, and blog posts, check out these links:

<div class="alert alert-block alert-success"><a id="resources"></a>

<h4>IBM documentation</h4>
 <ul>
 <li> <a href="https://wml-api-pyclient-dev-v4.mybluemix.net" target="_blank" rel="noopener no referrer">watson-machine-learning</a></li> 
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener noreferrer">CP4D 3.0</a></li>
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/deploy-batch-jobs.html" target="_blank" rel="noopener no referrer">Batch Deployments</a></li>
     <ul>
         <li> <a href="https://www.ibm.com/support/knowledgecenter/SSHGWL_3.0.0/wsj/wmls/wmls-deploy-python.html#deploy-batch" target="_blank" rel="noopener no referrer">Batch Deployments with the Python client</a></li>
         <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/deploy-batch-details.html" target="_blank" rel="noopener no referrer">Batch Deployment jobs</a></li>
    </ul>
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener noreferrer">Python Scripts</a></li>
 </ul>

<h4> IBM Samples</h4>
<br>
 <li> <a href="https://github.com/IBMDataScience/sample-notebooks" target="_blank" rel="noopener noreferrer">Sample notebooks</a></li>
 
<h4> Others</h4>
<br>
 <li> <a href="https://www.python.org" target="_blank" rel="noopener noreferrer">Official Python website</a></li>
 </div>

<hr>
Copyright ©2020 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>